# 用JPL历表——DE405算金星凌日

DE405所用时间为tdb的UT1

skyfield 包可以提供我们所需要的时间参数
最终从儒略日归算到 UT1


In [26]:
import numpy as np
from jplephem.spk import SPK
import de405
from jplephem import Ephemeris
import time


In [27]:
help(de405)

Help on package de405:

NAME
    de405 - JPL Planetary and Lunar Ephemeris DE405 for the jplephem package.

DESCRIPTION
    This ephemeris has been the basis for the Astronomical Almanac of the
    United States Naval Observatory since 2003 and powers the Jet Propulsion
    Laboratory's online HORIZONS service. Coordinates and velocities are
    provided for the major planets, the Sun, and Earth's Moon.
    
    :Name: DE405 (May 1997)
    :Years: 1600 through 2200
    :Planets: Yes
    :Sun/Moon: Yes
    :Nutations: Yes
    :Librations: Yes
    :Report: `Standish (1998) [PDF] <http://iau-comm4.jpl.nasa.gov/de405iom/de405iom.pdf>`_
    :Size: 54 MB
    
    The accuracy of this ephemeris is good; the position of the Earth or
    Mars, for example, should be accurate to within 2 km.  But the more
    recent `DE421 <http://pypi.python.org/pypi/de421>`_ ephemeris provides
    greater accuracy, especially with respect to the Moon, and you should
    use it instead if you are planning a spa

In [37]:
J2000_startJD = 2451545.0
JD1600 = 2305424.5
JD2200 = 2525008.5

delta_year = JD2200 - JD1600
delta_year

219584.0

In [40]:
# 计算金星凌日的时间——粗略计算——按天
# 金星-地球矢量 和 太阳-地球矢量比较
time_start = time.time()
Transit_date_raw = []
for delta_date in range(int(delta_year)):
    date = JD1600 + delta_date
    v = (eph.position('venus',date)).T[0]
#     v2 = kernel[0,2].compute(date) + kernel[2,299].compute(date)
    e = (eph.position('earthmoon', date) - eph.position('moon', date)*eph.earth_share).T[0]
#     e2 = kernel[0,3].compute(date) + kernel[3,399].compute(date)
    s = (eph.position('sun',date)).T[0]
#     s2 = kernel[0,10].compute(date)
    A = v - e ;B = s - e;
#     A2 = v2 - e2;B2 = s2 - e2;
    cos = A.dot(B)/(np.sqrt(A.dot(A))*np.sqrt(B.dot(B)))
#     cos2 = A2.dot(B2)/(np.sqrt(A2.dot(A2))*np.sqrt(B2.dot(B2)))

    if cos > 0.9999: # 金星进入日地视野锥面 —— 粗略计算
        if np.sqrt(A.dot(A)) < np.sqrt(B.dot(B)): # 金星在太阳和地球之间
            print('yeah!',date,cos)
            Transit_date_raw.append(date)
#             print(v,e,s)

print(time.time() - time_start)


yeah! 2317110.5 0.999969916322772
yeah! 2320030.5 0.9999738604082026
yeah! 2322949.5 0.9999138075697116
yeah! 2364408.5 0.9999777163404342
yeah! 2367328.5 0.9999937082872135
yeah! 2370248.5 0.9999111901389153
yeah! 2405866.5 0.9999803680752392
yeah! 2408786.5 0.9999601866275556
yeah! 2411705.5 0.9999222808947831
yeah! 2450245.5 0.9999082700965132
yeah! 2453164.5 0.9999492311484272
yeah! 2456084.5 0.999995003344639
yeah! 2459004.5 0.9999456787738176
yeah! 2494622.5 0.9999883843847766
yeah! 2497542.5 0.9999477779748337
yeah! 2500461.5 0.9999240596045589
49.30140686035156


In [54]:
# 一个儒略日 86400 SI秒，我们需要精确到分钟，就需要
1/86400 * 60
# 一分钟为0.0007 JDdays
# 但是其实我们并不能够获得精确的凌日开始/结束时间，所以我们给出相应的去掉秒数的那个时间就可以了

0.0006944444444444444

In [61]:
# 计算金星凌日，精确到分钟
time_start = time.time()
Transit_date_accurate_Start = []
Transit_date_accurate_End = []
bin_date = 0.001
for date_raw in Transit_date_raw:
    Transit_daytime = []
    for delta_date in range(1000): #从粗算的日子去加减0.5天（间隔1天），再取每个间隔为0.001day,获得开始和结束时间
        date = date_raw - 0.5 + delta_date*bin_date
        v = (eph.position('venus',date)).T[0]
    #     v2 = kernel[0,2].compute(date) + kernel[2,299].compute(date)
        e = (eph.position('earthmoon', date) - eph.position('moon', date)*eph.earth_share).T[0]
    #     e2 = kernel[0,3].compute(date) + kernel[3,399].compute(date)
        s = (eph.position('sun',date)).T[0]
    #     s2 = kernel[0,10].compute(date)
        A = v - e ;B = s - e;
    #     A2 = v2 - e2;B2 = s2 - e2;
        cos = A.dot(B)/(np.sqrt(A.dot(A))*np.sqrt(B.dot(B)))
    #     cos2 = A2.dot(B2)/(np.sqrt(A2.dot(A2))*np.sqrt(B2.dot(B2)))

        if cos > 0.999989193276346: # 金星进入日地视野锥面 —— 精确计算
            if np.sqrt(A.dot(A)) < np.sqrt(B.dot(B)): # 金星在太阳和地球之间
#                 print('yeah!',date,cos)
                Transit_daytime.append(date)
    if len(Transit_daytime) != 0:
        Transit_date_accurate_Start.append(np.min(np.array(Transit_daytime)))
        Transit_date_accurate_End.append(np.max(np.array(Transit_daytime)))
    print(Transit_date_accurate_Start)
    print(Transit_date_accurate_End,'end')
print(time.time() - time_start)

[2317110.685]
[2317110.753] end
[2317110.685, 2320030.13]
[2317110.753, 2320030.399] end
[2317110.685, 2320030.13]
[2317110.753, 2320030.399] end
[2317110.685, 2320030.13, 2364408.584]
[2317110.753, 2320030.399, 2364408.851] end
[2317110.685, 2320030.13, 2364408.584, 2367328.303]
[2317110.753, 2320030.399, 2364408.851, 2367328.558] end
[2317110.685, 2320030.13, 2364408.584, 2367328.303]
[2317110.753, 2320030.399, 2364408.851, 2367328.558] end
[2317110.685, 2320030.13, 2364408.584, 2367328.303, 2405866.586]
[2317110.753, 2320030.399, 2364408.851, 2367328.558, 2405866.749] end
[2317110.685, 2320030.13, 2364408.584, 2367328.303, 2405866.586, 2408786.088]
[2317110.753, 2320030.399, 2364408.851, 2367328.558, 2405866.749, 2408786.329] end
[2317110.685, 2320030.13, 2364408.584, 2367328.303, 2405866.586, 2408786.088]
[2317110.753, 2320030.399, 2364408.851, 2367328.558, 2405866.749, 2408786.329] end
[2317110.685, 2320030.13, 2364408.584, 2367328.303, 2405866.586, 2408786.088]
[2317110.753, 2320

# II. 把儒略日转化为ut1

In [42]:
# 使用skyfield包（注意需要科学上网下载deltat表）
from skyfield.api import load
import skyfield
import datetime

ts = skyfield.api.load.timescale()

[#################################] 100% deltat.preds


In [52]:
# Example
t = ts.tdb(jd=2305424.5)
tt = t.utc_datetime()
print(tt)



1599-12-08 23:59:17.817000+00:00


In [67]:
utc_start = []
utc_end = []
for i in range(len(Transit_date_accurate_Start)):
    
    for JDdate in Transit_date_accurate_Start:
        t_single = ts.tdb(jd = JDdate)
        utc_start.append(t_single.utc_datetime())

    for JDdate in Transit_date_accurate_End:
        t_single = ts.tdb(jd = JDdate)
        utc_end.append(t_single.utc_datetime())
    print(i+1)
    print(utc_start[i])
    print(utc_end[i])
    print('----------')


1
1631-12-07 04:25:41.817000+00:00
1631-12-07 06:03:37.017000+00:00
----------
2
1639-12-04 15:06:29.817000+00:00
1639-12-04 21:33:51.417000+00:00
----------
3
1761-06-06 02:00:15.415000+00:00
1761-06-06 08:24:44.215000+00:00
----------
4
1769-06-03 19:15:37.015000+00:00
1769-06-04 01:22:49.015000+00:00
----------
5
1874-12-09 02:03:08.217000+00:00
1874-12-09 05:57:51.417000+00:00
----------
6
1882-12-06 14:06:01.017000+00:00
1882-12-06 19:53:03.417000+00:00
----------
7
2004-06-08 05:12:51.015000+00:00
2004-06-08 11:14:17.415000+00:00
----------
8
2012-06-05 22:09:27.415000+00:00
2012-06-06 04:38:15.415000+00:00
----------
9
2117-12-11 00:11:48.417000+00:00
2117-12-11 05:18:31.617000+00:00
----------
10
2125-12-08 13:31:00.417000+00:00
2125-12-08 18:26:12.417000+00:00
----------


In [107]:
(2433282.50 - 2451545)/365.5

-49.965800273597814

In [92]:
np.cos(np.arctan((6.955E5)/(1.496E8)))

0.9999891932763468

In [31]:
eph.earth_share

0.01215058560962404